### Copyright (C) Microsoft Corporation.  
  
# Deploy regular ML R model in Azure AKS/ACI using AML SDK
Purpose: Before operationalization, we show how to develop and test the containerized scripts (o16n python script that invokes the user provided R scoring script).  
  
#### Authors

* **George Iordanescu** - *Initial work* - [Microsoft AI CAT](https://github.com/Azure/o16nRegularMLRmodelsUsingAzurek8s)

See also the list of [contributors](https://github.com/Azure/o16nRegularMLRmodelsUsingAzurek8s/contributors) who participated in this project.  
   
This notebook can be run in an AML SDK container on windows or linux:  
__on windows__:
```
(base) C:\Users\ghiordan\Documents>cd C:\repos\o16n_regular_ML_R_models_using_Azure_k8s
#docker run -it --volume %cd%:/workspace georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.17  /bin/bash
docker run -it -p 9001:8888 -v %cd%:/workspace georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.17 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=0.0.0.0 --port=8888 --no-browser --allow-root"  
  
# and then point local browser to:  
http://localhost:9001/?token=securitytoken_printed_by_jupyter_session_started_above
```
__on an Azure Linux VM (with port 9001 opened for Jupyter notebook server) named [yourVM]__:
```
[your_login_info]@[yourVM]:/repos/o16nRegularMLRmodelsUsingAzurek8s$ docker run -it -p 9001:8888 -v /datadrive01/prj/o16nRegularMLRmodelsUsingAzurek8s/:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.17 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=0.0.0.0 --port=8888 --no-browser --allow-root"  
   
# and then point local browser to:  
[yourVM].eastus2.cloudapp.azure.com:9001/?token=eeb532b4481b8aa0dcee646c72f48a6dc162a38fb314be9d  
```

Here we use the experimentation (__e13n__) infrastructure in AML SDK to build a docker image and test score.py script. This docker image alows running R code from python and is the closest proxy to the operationalization (__o16n__) docker image one will get in the next notebooks, where we operationalize the R model. The created docker image is __not__ identical to the o16n image because AML SDK does not allow BYOD (bring your own docker) scenario yet.

The score.py script is written in python, but it has an R session created via rpy2. The R model is run via four interactions with an R session:
 - The init() function in score.py passes the R model file name to the R session which then loads the R model.  
 - The run() function in score.py passes the jsoned data to be scored to the R session with the model loaded above. jsoned data to be scored are then used with a full R scoring script using rpy2.robjects.r().  

Main steps:  
* Run score.py script (and the real R scoring script) in a docker image.
* Create artifacts for deployment:   
  - scoring script file in the project folder (variable __score_script_filename__)  
  - conda dependency file (adds R and desired packages to the base docker image)

* This covers strictly post e13n steps, so it assumes the existence of R model file (saved as an rds file on disk). This will be registered here so that we can use Model.get_model_path() function inside the init() funstion of the scoring script. See [this](https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/docs/get_model_path.md) for details about encapsulating the model path in the 016n docker image.

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Check core SDK version number

In [2]:
# Check core SDK version number, os info and current wd
import azureml.core
print("SDK version:", azureml.core.VERSION)

#azure ml files location
azureml.__file__

import platform
platform.platform()

import os
os.getcwd()

SDK version: 1.0.17


'/opt/conda/envs/aml-sdk-conda-env/lib/python3.6/site-packages/azureml/__init__.py'

'Linux-4.9.125-linuxkit-x86_64-with-debian-9.5'

'/workspace/code/amlsdk_operationalization'

In [3]:
# import utility functions like project config params

import sys, os

def add_path_to_sys_path(path_to_append):
    if not (any(path_to_append in paths for paths in sys.path)):
        sys.path.append(path_to_append)

auxiliary_files_dir = os.path.join(*(['.', 'src']))

paths_to_append = [os.path.join(os.getcwd(), auxiliary_files_dir)]
[add_path_to_sys_path(crt_path) for crt_path in paths_to_append]

import o16n_regular_ML_R_models_utils
prj_consts = o16n_regular_ML_R_models_utils.o16n_regular_ML_R_models_consts()

[None]

### use existing dotenv file (created in previous notebook) to load sensitive info

In [4]:
%load_ext dotenv
dotenv_file_path = os.path.join(*(prj_consts.DOTENV_FILE_PATH))

#show .env file path
dotenv_file_path

'../../not_shared/general.env'

#### Define filename and directory variables


In [5]:
r_model_file_name = prj_consts.R_MODEL_FILE_NAME
r_model_AML_name = prj_consts.R_MODEL_AML_NAME
conda_dependencies_filename = prj_consts.R_MODEL_CONDA_DEPENDENCIES_FILE_NAME
score_script_filename = prj_consts.SCORE_SCRIPT_FILE_NAME
experiment_name = 'ghiordan_test_R_scoring_script'

workspace_config_dir = os.path.join(*(prj_consts.AML_WORKSPACE_CONFIG_DIR))
workspace_config_file = prj_consts.AML_WORKSPACE_CONFIG_FILE_NAME
workspace_config_dir

experiment_dir = os.path.join(*(prj_consts.AML_EXPERIMENT_DIR))
crt_dir = os.path.join(os.getcwd(), os.path.join(*([experiment_dir])))
if not (os.path.isdir(crt_dir)): os.mkdir(crt_dir) 

# make sure exp name is within required limits
len(experiment_name)

R_artifacts_dir = os.path.join(os.getcwd(), os.path.join(*(prj_consts.R_MODEL_DIR)))
print('Will o16n R model from directory {}'.format(R_artifacts_dir))

'../../not_shared'

30

Will o16n R model from directory /workspace/code/amlsdk_operationalization/../R_experimentation


#### Use the AML SDK workspace (ws) created and documented as a json file in previous notebook

Initialize a workspace object from persisted configuration.

In [6]:
from azureml.core import Workspace

# path arg is:
#   - a file path which explictly lists aml_config subdir for function from_config() 
#   - a dir path with a silently added <<aml_config>> subdir for function write_config(). 
ws = Workspace.from_config(path=os.path.join(os.getcwd(), 
                                             os.path.join(*([workspace_config_dir, 'aml_config', workspace_config_file]))))
print(ws.name, ws.resource_group, ws.location, ws.subscription_id[0], sep = '\n')

Found the config file in: /workspace/not_shared/aml_config/aml_ws_config.json
ghiordanregularrrealtimews
ghiordanRo16n1rsg02
eastus2
e


## Register Model

You can add tags and descriptions to your models. Note you do not need to have the r model .rds file in the current directory.  The below call registers that file in the workspace as a model with name defined by __r_model_AML_name__ variable.  
  
Using tags, you can track useful information such as the name and version of the machine learning library used to train the model. Note that tags must be alphanumeric.

In [7]:
#show model exists at the expected location 
!ls -l {os.path.join(R_artifacts_dir, r_model_file_name)}

-rwxr-xr-x 1 root root 19041 Jul 25  2018 /workspace/code/amlsdk_operationalization/../R_experimentation/ksvm_model01.rds


In [8]:
from azureml.core.model import Model
register_model = True
if register_model:
    model = Model.register(model_path = os.path.join(R_artifacts_dir, r_model_file_name),
                           model_name = r_model_AML_name,
                           tags = {'language': 'R', 'type': 'TC_kSVM'},
                           description = 'my R model',
                           workspace = ws)
    
    print(model.name, model.description, model.version, model.tags, sep = '\t')

Registering model trained_r_model
trained_r_model	my R model	1	{'language': 'R', 'type': 'TC_kSVM'}


You can explore the registered models within your workspace and query by tag. Models are versioned. If you call the register_model command many times with same model name, you will get multiple versions of the model with increasing version numbers.   

For demo purposes, we choose v1 as the model used for deployment.

In [9]:
best_r_model = None

for m in Model.list(ws, tags={'type': 'TC_kSVM'}):
# for m in r_models:
    print("Name:", m.name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)
    if ((m.name==r_model_AML_name) and (m.version==1) and (m.description=='my R model')):
        best_r_model = m

Name: trained_r_model 	Version: 1 	Description: my R model {'language': 'R', 'type': 'TC_kSVM'}


In [10]:
print(best_r_model.name, best_r_model.description, best_r_model.version, sep = '\t')

trained_r_model	my R model	1


## Create Experiment


In [11]:
from azureml.core import Experiment

exp = Experiment(workspace = ws, name = experiment_name)

## Create info for the VM compute target

Attach a remote Linux VM. This is usually used as a remote docker commpute target for experimentation, but we are using it here to test our dockerized score script used for deploymnet. 
Create a Linux DSVM in Azure. Make sure you use the Ubuntu flavor, NOT CentOS.      

In [12]:
compute_target_name = 'ghiordanXRgpuvm'

In [13]:
%dotenv  $dotenv_file_path

from azureml.core.compute import ComputeTarget, RemoteCompute 
attach_config = RemoteCompute.attach_configuration(address=os.getenv('COMPUTE_CONTEXT_VM_FQDN'),
                                                   ssh_port=os.getenv('COMPUTE_CONTEXT_VM_SSH_PORT'),
                                                   username=os.getenv('COMPUTE_CONTEXT_VM_USER_NAME'),
                                                   password=os.getenv('COMPUTE_CONTEXT_VM_PWD')
                                                   # If using ssh key
                                                   #private_key_file="path_to_a_file",
                                                   #private_key_passphrase="some_key_phrase"
                                                  )
attached_dsvm_compute = ComputeTarget.attach(workspace=ws, name=compute_target_name, attach_configuration=attach_config)

attached_dsvm_compute.wait_for_completion(show_output=True)   

SucceededProvisioning operation finished, operation "Succeeded"


In [14]:
# see if the compute target exists in the workspace

from azureml.core.compute import DsvmCompute

for crt_dsvm in DsvmCompute.list(ws):
    if (compute_target_name==crt_dsvm.name):    
        print(crt_dsvm.name, crt_dsvm.type, crt_dsvm.address)
    else:
        print(crt_dsvm.name, crt_dsvm.type)

ghiordanXRgpuvm VirtualMachine ghiordanxray05vm.eastus.cloudapp.azure.com


## Create scoring script

Use `%%writefile` magic to write o16n `score.py` file  that embeds the user-prvided R scoring script.

In [15]:
%%writefile {os.path.join(experiment_dir, score_script_filename)} 


import pickle
import json
from azureml.core.model import Model
import rpy2
import rpy2.robjects as robjects
import timeit
import logging

R_MODEL_AML_NAME = 'trained_r_model'


def init():
    from rpy2.rinterface import R_VERSION_BUILD
    print('rpy2 version {};  R version {}'.format(rpy2.__version__, R_VERSION_BUILD))
    
    print('R model AML name: {}'.format(Model.get_model_path(model_name='trained_r_model')))
    
    global model
    # note here "best_model" is the name of the model registered under the workspace
    # this call should return the path to the model.pkl file on the local disk.
    model_path = Model.get_model_path(model_name =  'trained_r_model')
    # deserialize the model file back into a sklearn model
    robjects.globalenv['model_path'] = model_path    
    # model_path = robjects.StrVector( 'ksvm_model.rds')
    robjects.r('''
            format_proc_time <- function(proc_time_diff){
                
                as.data.frame(t(as.matrix(format(round(proc_time_diff*1000, 2), nsmall = 2))))[, 
                                                            c('user.self', 'sys.self', 'elapsed')]
            }
            library(kernlab)
            library(jsonlite)
            svm_model = readRDS({model_path})
            ''')
    print('AML o16n init() function: SVM model loaded.')

# note you can pass in multiple rows for scoring
def run(aml_jsoned_data):
    logger = logging.getLogger("AML_o16n_run_function")
#     print('Entering run() function')
    try:
        start_time = timeit.default_timer()
#         data = json.loads(raw_data)['data']
        data = json.loads(aml_jsoned_data)['data']
        robjects.globalenv['r_data_to_score'] = data  
        python_to_R_time = timeit.default_timer()
        r_messages = robjects.r('''
                start_time_r = proc.time()
                
                r_data_to_score=jsonlite::fromJSON(r_data_to_score[[1]])
                json_to_df_time_r = proc.time()
                
                scores = kernlab::predict(svm_model,r_data_to_score, type = "p")
                end_time_r = proc.time()
                
                # report total time and json to df time
                time_df = rbind(format_proc_time(end_time_r - start_time_r),
                                format_proc_time(json_to_df_time_r - start_time_r))
                rownames(time_df)=c('all_r_time','json_to_df_time')    
                
                # combine scores and time dataframes in a list
                returned_list = list(as.data.frame(scores),time_df)
                names(returned_list)=c('r_scores', 'r_times')
                
                scores = jsonlite::toJSON(returned_list)
                #print('Exiting R.')
                ''')
        before_R_to_python_time = timeit.default_timer()
        
        jsoned_scores = (robjects.r['scores'])[0]
        end_time = timeit.default_timer()
        
#         logger.info("Predictions: {0}".format(jsoned_scores))
#         print('Exiting run() function')
        return json.dumps({'python_scores': jsoned_scores, 
                           'python_times': json.dumps(
                               {'all_p_time':'{} ms'.format(round((end_time-start_time)*1000, 2)),
                                           'python_to_R_time':'{} ms'.format(round((python_to_R_time-start_time)*1000, 2)),
                                           'R_to_python_time':'{} ms'.format(round((end_time-before_R_to_python_time)*1000, 2))}
                           )
                          })

    except Exception as e:
        result = str(e)
        return json.dumps({"AML o16n run() function: error": result})
    
def main():
    import numpy as np
    import pandas as pd
    
    n_samples = 100

    raw_data = 2 * np.random.random_sample((n_samples, 2)) - 1
    aml_jsoned_data =  json.dumps({'data': json.dumps(raw_data.tolist())})
  
    init()
    response = run(aml_jsoned_data)
#     print(json.loads(response))
#     print( json.loads(json.loads(response)['python_scores']) )
    
    print( pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_scores']) )
    print( pd.DataFrame.from_records(json.loads(json.loads(response)['python_scores'])['r_times']) )
    for k, v in json.loads(json.loads(response)['python_times']).items():
        print(v, k)

    print('Exited main() function')
    
if __name__== "__main__":
    main()
    

Overwriting ../../temp/score_script.py


## Configure a Docker run with new conda environment on the VM  
You can execute in a Docker container in the VM. If you choose this route, you don't need to install anything on the VM yourself. Azure ML execution service will take care of it for you.

### Configure a run using a custom Docker  image & user-managed environment
You can also build a custom Docker image, and execute script in it without building a new conda environment. The Docker file for the below `ninghai/azureml:0.3` image is simply using the `miniconda3` base image from Continuum.io, plus essential build tools, plus Azure ML SDK. This image is [hosted in Docker Hub](https://hub.docker.com/r/ninghai/azureml). But you can also host it in a private registry like ACR (Azure Container Registry).

```dockerfile
FROM continuumio/miniconda3
RUN apt-get update && apt-get install -y build-essential
RUN pip install --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/rc/D72EBD8984244C659BDE1CFECDC3435 azureml-requirements
```

You can of course use the Dockerfile to install additional libraries that cannot be conda-installed or pip-installed.

In [16]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

run_config = RunConfiguration(framework = "python")
run_config.target = attached_dsvm_compute.name

# Use Docker in the remote VM
run_config.environment.docker.enabled = True

run_config.environment.docker.base_image = 'continuumio/miniconda3:4.5.12'
print('Base Docker image is:', run_config.environment.docker.base_image )

# Ask system to provision a new one based on the conda_dependencies.yml file
run_config.environment.python.user_managed_dependencies = False

Base Docker image is: continuumio/miniconda3:4.5.12


#### Below is the crux of the project:
> Instead of install.packages() using a live R session, we use conda and pip install to add R and desired packages to a conda environment that will run in our docker container!

In [17]:
# create the conda env yml file
from azureml.core.conda_dependencies import CondaDependencies

conda_dep = CondaDependencies()

def add_conda_items(list_of_items, item_add_method):
    for current_item in list_of_items:
        item_add_method(current_item)
    
add_conda_items(['3.7.0'], getattr(conda_dep, 'set_python_version')) #'3.6.5' 
add_conda_items(['r', 'conda-forge', 'anaconda'], getattr(conda_dep, 'add_channel'))
add_conda_items(['r-base', 'r-proc', 'r-jsonlite', 'r-kernlab', 'rpy2', 'pandas', 'gfortran_linux-64'], 
                getattr(conda_dep, 'add_conda_package')) #'rpy2==2.8.6'
# add_conda_items(['some_pip_installable_R_package'], getattr(conda_dep, 'add_pip_package'))

In [18]:
run_config.environment.docker.gpu_support = False
run_config.environment.docker.shared_volumes

True

#### For demo purposes we also show how a generic (i.e. not SDK created) conda env file can be used in SDK

First show the content of the conda dep object above, so that we can create a clone of it.

In [19]:
conda_dep.serialize_to_string()
conda_dep.save_to_file(base_directory = os.getcwd() , 
                       conda_file_path=os.path.join(*[experiment_dir, conda_dependencies_filename]))
! cat {os.path.join(os.getcwd(), os.path.join(*[experiment_dir, conda_dependencies_filename]))}

not_SDK_created_conda_env_file = 'not_SDK_created_conda_env_file.yml'

'# Conda environment specification. The dependencies defined in this file will\r\n# be automatically provisioned for runs with userManagedDependencies=False.\r\n\n# Details about the Conda environment file format:\r\n# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually\r\n\nname: project_environment\ndependencies:\n  # The python interpreter version.\r\n  # Currently Azure ML only supports 3.5.2 and later.\r\n- python=3.7.0\n\n- pip:\n    # Required packages for AzureML execution, history, and data preparation.\r\n  - azureml-defaults\n- r-base\n- r-proc\n- r-jsonlite\n- r-kernlab\n- rpy2\n- pandas\n- gfortran_linux-64\nchannels:\n- r\n- conda-forge\n- anaconda\n'

'../../temp/conda_dependencies01.yml'







name: project_environment
dependencies:


- python=3.7.0

- pip:

  - azureml-defaults
- r-base
- r-proc
- r-jsonlite
- r-kernlab
- rpy2
- pandas
- gfortran_linux-64
channels:
- r
- conda-forge
- anaconda


In [20]:
%%writefile ./{not_SDK_created_conda_env_file}

name: ml_conda_env2

channels:
- r
- conda-forge
- anaconda

dependencies:
- python=3.7.0
- r-base
- r-proc
- r-jsonlite
- r-kernlab
- rpy2
- pandas
- gfortran_linux-64
- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

Overwriting ./not_SDK_created_conda_env_file.yml


Now, one can use either of the commands below to set the conda dependencies either from a yml file, or from the CondaDependencies conda_dep object created in memory above 

In [21]:
# run_config.environment.python.conda_dependencies = CondaDependencies(not_SDK_created_conda_env_file)
run_config.environment.python.conda_dependencies = conda_dep

#### Script below can be used to replace proper score_script_filename, for debug purposes

In [22]:
# %%writefile $experiment_dir/$score_script_filename 

# import subprocess
# subprocess.call(['pwd'])
# subprocess.call(['ls', '-l'])
# subprocess.call(['conda', 'env','list'])

### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the `conda_dependencies.yml` file on top of the base image, create a container and then execute the script in the container.

## Configure & Run

In [23]:
%%time 

from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = experiment_dir, script = score_script_filename, run_config = run_config)
run = exp.submit(src)
run.wait_for_completion(show_output = True)
run.get_portal_url()



RunId: ghiordan_test_R_scoring_script_1552066598_7071fb24

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Logging experiment preparation status in history service.
Running: ['sudo', 'docker', 'build', '-f', 'azureml-setup/Dockerfile', '-t', 'azureml/azureml_6a5da0d9647c03aec276af894c27d83b', '.']
Sending build context to Docker daemon  179.2kB
Step 1/13 : FROM continuumio/miniconda3:4.5.12
4.5.12: Pulling from continuumio/miniconda3
05d1a5232b46: Pulling fs layer
d974dd5eb235: Pulling fs layer
2de22c73730e: Pulling fs layer
444f639f1b28: Pulling fs layer
c1b600cb48ba: Pulling fs layer
444f639f1b28: Waiting
c1b600cb48ba: Waiting
d974dd5eb235: Verifying Checksum
d974dd5eb235: Download complete
05d1a5232b46: Verifying Checksum
05d1a5232b46: Download complete
444f639f1b28: Verifying Checksum
444f639f1b28: Download complete
c1b600cb48ba: Verifying Checksum
c1b600cb48ba: Download complete
2de22c73730e: Verifying Checksum
2de22c73730e: Download c

binutils_impl_linux- | 16.5 MB   | ########## | 100% 
libxml2-2.9.8        | 2.0 MB    | ########## | 100% 
r-bh-1.66.0_1        | 10.0 MB   | ########## | 100% 
rpy2-2.9.1           | 268 KB    | ########## | 100% 
r-rcolorbrewer-1.1_2 | 58 KB     | ########## | 100% 
graphite2-1.3.13     | 109 KB    | ########## | 100% 
pytz-2018.9          | 229 KB    | ########## | 100% 
r-mgcv-1.8_24        | 2.5 MB    | ########## | 100% 
libcurl-7.64.0       | 586 KB    | ########## | 100% 
r-labeling-0.3       | 64 KB     | ########## | 100% 
r-assertthat-0.2.0   | 57 KB     | ########## | 100% 
bwidget-1.9.11       | 113 KB    | ########## | 100% 
fontconfig-2.13.0    | 291 KB    | ########## | 100% 
bzip2-1.0.6          | 415 KB    | ########## | 100% 
pthread-stubs-0.4    | 5 KB      | ########## | 100% 
numpy-base-1.16.2    | 4.3 MB    | ########## | 100% 
xz-5.2.4             | 366 KB    | ########## | 100% 
gxx_impl_linux-64-7. | 18.7 MB   | ########## | 100% 
sqlite-3.26.0        | 1.9 M

  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
Successfully built pathspec pyyaml pycparser

#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_93ff344e89b19084da4a6791da55a8c6
#
# To deactivate an active environment, use:
# > source deactivate
#



Removing intermediate container 974ad33db431
 ---> 953f99a0413b
Step 9/13 : ENV PATH /azureml-envs/azureml_93ff344e89b19084da4a6791da55a8c6/bin:$PATH
 ---> Running in 7716a8b1a68c
Removing intermediate container 7716a8b1a68c
 ---> ddf5c69d29be
Step 10/13 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_93ff344e89b19084da4a6791da55a8c6/lib:$LD_LIBRARY_PATH
 ---> Running in 19642eb0e362
Removing intermediate container 19642eb0e362
 ---> 8691a67738a4
Step 11/13 : COPY azureml-setup/spark_cache.py azureml-setup/log4j.properties /azureml-setup/
 ---> a1312bb96a6c
Step 12/13 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit "--repositories" "https://mmlspark.azureedge.net/maven" "--packages" "com.microsoft.ml.spark:mmlspark_2.11:0.12" /azureml-setup/spark_cache.py'; fi
 ---> Running in 3af25cd2ae13
Removing intermediate container 3af25cd2ae13
 ---> f7fe41cf27da
Step 13/13 : CMD ["bash"]
 ---> Running in 7a5bf275499a
Removing intermediate container 7a5bf275499a
 ---> 1e9d80

In [24]:
# # to recover a previous run
# run = Run(exp, 'runID')

print(run.get_portal_url())

# # to get more details
# run.get_details_with_logs()

https://mlworkspace.azure.ai/portal/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/ghiordanRo16n1rsg02/providers/Microsoft.MachineLearningServices/workspaces/ghiordanregularrrealtimews/experiments/ghiordan_test_R_scoring_script/runs/ghiordan_test_R_scoring_script_1552066598_7071fb24


#### We are now ready for deployment. In the next notebook we will deploy image as web service on Azure Container Instance


In [25]:
!jupyter nbconvert --to html 010_RegularR_RealTime_test_score_in_docker.ipynb

[NbConvertApp] Converting notebook 010_RegularR_RealTime_test_score_in_docker.ipynb to html
[NbConvertApp] Writing 357069 bytes to 010_RegularR_RealTime_test_score_in_docker.html
